In [7]:
from openpyxl import load_workbook
from openpyxl import Workbook
from openpyxl.styles import NamedStyle, Font, PatternFill, Alignment
import os

def check_fingerprint(wb, matrnr):
    ws = wb["1) Einkaufsliste"]
    if ws["AA1"].value == int(matrnr):
        return "OK"
    else:
        return "NOK"

def check_exercise_1_1_1(wb):
    # Check formatting of cells A1:B2
    ws = wb["1) Einkaufsliste"]
    points = 0
    # Check if A1 and B1 are merged
    for rng in ws.merged_cells.ranges:
        if str(rng) == "A1:B1":
            points += 1
    # Check if A1, A2, B2 are all bold
    if ws["A1"].font.b and ws["A2"].font.b and ws["B2"].font.b:
        points += 1
    # Check if A1, A2, B2 background color is FH-Mint
    if ws["A1"].fill.start_color.index == ws["A2"].fill.start_color.index == ws["A2"].fill.start_color.index == "FF00B1AC":
        points += 1
    # Check if A1 has font-size 16
    if ws["A1"].font.size == 16:
        points += 1
    # Check if A1, A2, B2 are aligned centrally
    if ws["A1"].alignment.horizontal == ws["A2"].alignment.horizontal == ws["B2"].alignment.horizontal == "center":
        points += 1

    return points

def check_exercise_1_1_2(wb):
    # Check the width of Column A. Should be 20. Difficult to check. So check, if width of A is bigger than width of C
    ws = wb["1) Einkaufsliste"]
    if ws.column_dimensions["A"].width >= 20:
        return 1
    else:
        return 0

def check_exercise_1_1_3(wb):
    # Check, if the numberformat in cells B2, B3, B4 is currency
    ws = wb["1) Einkaufsliste"]
    if ws["B3"].number_format == r'_-* #,##0.00\ "€"_-;\-* #,##0.00\ "€"_-;_-* "-"??\ "€"_-;_-@_-': # Excel dt "Buchhaltung"
        return 1
    if ws["B3"].number_format == r'#,##0.00\ "€"':                                                  # Excel dt "Währung"
        return 1
    if ws["B3"].number_format == r'0.00 "€"':
        return 1

    return 0

def check_exercise_1_1_4(wb):
    # Check, if the border is given above the sums
    ws = wb["1) Einkaufsliste"]
    points = 0
    cell1 = ws["B" + str(ws.max_row - 3)]
    cell2 = ws["B" + str(ws.max_row - 2)]
    cell3 = ws["B" + str(ws.max_row - 1)]
    cell4 = ws["B" + str(ws.max_row)]
    if cell1.border.bottom.style == "medium" or cell2.border.top.style == "medium":
        points += 1
    if cell3.border.bottom.style == "medium" or cell4.border.top.style == "medium":
        points += 1

    return points

def check_exercise_1_1_5(wb, wb_val):
    # Check, if the calculations are correct
    ws = wb_val["1) Einkaufsliste"]
    points = 0
    l_prices = []
    for row in range(3, ws.max_row - 2):
        price = ws["B" + str(row)].value
        l_prices.append(price)
    if sum(l_prices) == round(ws["B" + str(ws.max_row - 2)].value, 2):
        points += 1
    if ws["A" + str(ws.max_row - 1)].number_format == '"MwSt. "0%':
        if round(ws["B" + str(ws.max_row - 1)].value, 2) == round(ws["B" + str(ws.max_row - 2)].value * ws["A" + str(ws.max_row - 1)].value, 2):
            points += 1
    if not ws["B" + str(ws.max_row - 2)].number_format == "General" and not ws["B" + str(ws.max_row - 1)].number_format == "General" and not ws["B" + str(ws.max_row)].number_format == "General":
        if round(ws["B" + str(ws.max_row - 2)].value + ws["B" + str(ws.max_row - 1)].value, 2) == round(ws["B" + str(ws.max_row)].value, 2):
            points += 1

    # Check formulas
    ws = wb["1) Einkaufsliste"]
    cell1 = "B" + str(ws.max_row - 2)
    cell2 = "B" + str(ws.max_row - 1)
    cell3 = "B" + str(ws.max_row)
    if ws[cell3].value == "=" + cell1 + "+" + cell2 or ws[cell3].value == "=" + cell2 + "+" + cell1 or ws[cell3].value == "=SUM(" + cell1 + ":" + cell2 + ")":
        points += 1

    if ws[cell1].value == "=SUM(B3:B" + str(ws.max_row - 3) + ")":
        points += 1
    
    return points

def check_exercise_1_1(matrnr, wb, wb_val):
    points = [matrnr]
    points.append(check_fingerprint(wb, matrnr))
    points.append(check_exercise_1_1_1(wb))
    points.append(check_exercise_1_1_2(wb))
    points.append(check_exercise_1_1_3(wb))
    points.append(check_exercise_1_1_4(wb))
    points.append(check_exercise_1_1_5(wb, wb_val))
    
    
    return points

def check_excel_file(filename):
    if filename[7:] == ".xlsx":
        try:
            matrnr = int(filename[:7])
            wb = load_workbook(filename)
            wb_val = load_workbook(filename, data_only = True)
            points = check_exercise_1_1(matrnr, wb, wb_val)
            return points
        except:
            print("Problems with ", filename, "! Cannot be opened by openpyxl")
    return 0    

def check_ExcelFiles_in_Abgaben():
    l_points = []
    if os.path.exists("Abgaben"):
        os.chdir("Abgaben")
        for subdir in os.listdir():
            if subdir != ".DS_Store":
                os.chdir(subdir)
                filename = os.listdir()[0]
                points = check_excel_file(filename)
                l_points.append(points)
                os.chdir("..")
        os.chdir("..")
    else:
        print("No directory named 'Abgaben'.")
    return l_points

def result_into_file(l_points):
    wb = Workbook()
    ws = wb.active
    titles = ["MatrNr", "Fingerprint", "Format", "Spalte", "Zahlenformat", "Rahmen", "Formeln"]
    ws.append(titles)
    for points in l_points:
        ws.append(points)
    
    # Some formatting
    highlight = NamedStyle(name = "highlight")
    highlight.font = Font(bold = True, color = "FFFFFF")
    highlight.fill = PatternFill(fgColor = "00b1ac", fill_type = "solid")
    highlight.alignment = Alignment(horizontal = "center")
    wb.add_named_style(highlight)
    
    for row in ws["A1:G1"]:
        for cell in row:
            cell.style = "highlight"
            
    wb.save("Excel_01.xlsx")
    
    

l_points = check_ExcelFiles_in_Abgaben()
result_into_file(l_points)


In [3]:
os.chdir(".")
print(os.getcwd())

/Users/jacobs/GIT/PC_Praktikum/01_Excel
